##### Auteur: Antoine Cadiou

# Attention: pour fonctionner il faut récupérer les données sur ces liens: 
### https://www.kaggle.com/antocad/labels-siim-isic-224/settings
### https://www.kaggle.com/arroqc/siic-isic-224x224-images
### www.kaggle.com/dataset/2e87262f75481531a0ed807cfbe69fbb4ff7b43c39fc5b3a6c812bc27f026942
#### et changer les chemins des données dans le code

In [ ]:
from __future__ import division
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.utils.vis_utils import plot_model as plot
from keras.optimizers import SGD
from keras.optimizers import *
from keras.layers import *        
from keras.applications.vgg16 import VGG16
import keras

import numpy as np
import pandas as pd
import os, cv2
import matplotlib.pyplot as plt
from scipy import ndimage

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from tensorflow.keras import datasets
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import optimizers, layers, regularizers

In [ ]:
CFG = dict(
    DEVICE = 'GPU',
    
    train_size = 0.8,
    random_seed = 42, #None ou int
    
    img_size = 256,
    epochs = 100,
    batch_size = 32,
    
    lr_start = 0.000006,
    lr_max = 0.00000145,
    lr_min = 0.000001,
    lr_rampup = 5,
    lr_sustain = 0,
    lr_decay = 0.85,
    optimizer = 'adam',
    label_smooth_fac  =   0.05,
    
    net_count = 1,
    tta_steps = 1,
)

if CFG['DEVICE'] == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        CFG['DEVICE'] = "GPU"

if CFG['DEVICE'] != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if CFG['DEVICE'] == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
#on lit les données csv
train_original = pd.read_csv('../input/labels-siim-isic-224/train.csv', dtype=str)
#on construit un dataframe réduit, plus équilibré
part_true = train_original[train_original["target"] == '1']
part_false = train_original[train_original["target"] == '0'].sample(len(part_true) * 3)
train_balanced = pd.concat([part_true, part_false])
#on shuffle les lignes du dataframe
if CFG['random_seed']!=None:
    np.random.seed(CFG['random_seed'])
cols = train_balanced.columns
idshuffle = np.arange(len(train_balanced))
np.random.shuffle(idshuffle)
df = pd.DataFrame(np.array(train_balanced)[idshuffle])
df.columns = cols
#on construit la colonne filename + nettoyage
df['filename'] = df['image_name']+'.png'
df['target'] = df['benign_malignant']
df = df.drop(columns=['image_name', 'patient_id','sex','age_approx','anatom_site_general_challenge','diagnosis','benign_malignant'])
df = df[['filename', 'target']]
df.info()
df.head()

In [ ]:
def dullrazor(img, lowbound=20, filterstruc=7, inpaintmat=3):
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #applying a blackhat
    filterSize =(filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)
    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)
    return img_final

def getSegmentation(img):
    #gaussian filter
    img = cv2.GaussianBlur(img,(7,7),0)
    #gray threshold
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    #Morphological reconstruction
    def morph(I, M):
        struc = np.ones((3,3))
        r = np.sum(I)
        s = 0
        while s != r:
            s = r
            dilated = ndimage.binary_dilation(M, structure=struc)
            M = np.logical_and(I, dilated)
            r = np.sum(M)
        return M
    M = np.zeros((CFG['img_size'], CFG['img_size']))
    rond = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(101,101))
    m = int(CFG['img_size']/2)
    M[m-50:m+51, m-50:m+51] = rond
    mask = morph(mask, M).astype('uint8')
    # noise removal
    kernel = np.ones((7,7),np.uint8)
    mask = ndimage.binary_dilation(mask, structure=kernel)
    mask = ndimage.binary_opening(mask, structure=np.ones((11,11),np.uint8))
    return mask

def preprocess(input_img):
    img = (input_img.copy()).astype('uint8')
    plt.imshow(img)
    plt.title("image originale")
    plt.show()
    # white balance for every channel independently
    def wb(channel, perc = 0.05):
        mi, ma = (np.percentile(channel, perc), np.percentile(channel,100.0-perc))
        channel = np.uint8(np.clip((channel-mi)*255.0/(ma-mi), 0, 255))
        return channel
    img  = np.dstack([wb(channel, 0.05) for channel in cv2.split(img)])
    #dullrazor
    img = dullrazor(img)
    plt.imshow(img)
    plt.title("image après préprocessing")
    plt.show()
    mask = getSegmentation(img).astype('uint8')
    res = cv2.cvtColor(mask,cv2.COLOR_GRAY2RGB)
    plt.imshow(mask)
    plt.title("méthode 1")
    plt.show()
    return img_to_array(img)

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale = 1./255.,
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = df[6:7],
    directory = "../input/siic-isic-224x224-images/train", 
    x_col = "filename",
    y_col="target",
    class_mode="categorical",
    shuffle=False,
    target_size = (CFG['img_size'], CFG['img_size']), 
    batch_size = CFG['batch_size'],
)

In [ ]:
#On parcours toutes les images et on récupère les segmentations dans un tableau.
#Enfin, on peut exporter ce tableau dans un fichier pickle, pour les etudier dans un autre notebook
data_list = np.zeros((1,224,224,3))
label_list = np.zeros((1,2))
batch_index = 0

while batch_index <= test_generator.batch_index:
    data,label = test_generator.next()
    data_list = np.vstack((data_list, data))
    label_list = np.vstack((label_list, label))
    batch_index = batch_index + 1
    
data_list = data_list[1:]
label_list = label_list[1:]

In [ ]:
# import pickle
# with open('imgs3.pickle', 'wb') as handle:
#     pickle.dump(data_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('labels3.pickle', 'wb') as handle:
#     pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# U NET

In [ ]:
def BCDU_net_D3(input_size = (256,256,1)):
    N = input_size[0]
    img_input = Input(input_size)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(img_input)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    # D1
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)     
    conv4_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4_1 = Dropout(0.5)(conv4_1)
    # D2
    conv4_2 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop4_1)     
    conv4_2 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4_2)
    conv4_2 = Dropout(0.5)(conv4_2)
    # D3
    merge_dense = concatenate([conv4_2,drop4_1], axis = 3)
    conv4_3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge_dense)     
    conv4_3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4_3)
    drop4_3 = Dropout(0.5)(conv4_3)

    up6 = Conv2DTranspose(256, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(drop4_3)
    up6 = BatchNormalization(axis=3)(up6)
    up6 = Activation('relu')(up6)

    x1 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), 256))(drop3)
    x2 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), 256))(up6)
    merge6  = concatenate([x1,x2], axis = 1) 
    merge6 = ConvLSTM2D(filters = 128, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal')(merge6)

    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2DTranspose(128, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv6)
    up7 = BatchNormalization(axis=3)(up7)
    up7 = Activation('relu')(up7)

    x1 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), 128))(conv2)
    x2 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), 128))(up7)
    merge7  = concatenate([x1,x2], axis = 1) 
    merge7 = ConvLSTM2D(filters = 64, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge7)

    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2DTranspose(64, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv7)
    up8 = BatchNormalization(axis=3)(up8)
    up8 = Activation('relu')(up8)    

    x1 = Reshape(target_shape=(1, N, N, 64))(conv1)
    x2 = Reshape(target_shape=(1, N, N, 64))(up8)
    merge8  = concatenate([x1,x2], axis = 1) 
    merge8 = ConvLSTM2D(filters = 32, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge8)    

    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv9 = Conv2D(1, 1, activation = 'sigmoid')(conv8)

    model = Model(img_input, conv9)
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = BCDU_net_D3(input_size = (256,256,3))#remettre 256 ?
model.load_weights("../input/weightsbcdunetd3/bcdunet_melanoma.hdf5")
res = model.predict_generator(
    test_generator
)